In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_09-14-39"))

In [ ]:
rdf.head()

In [ ]:
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, no profits, mixed curves. Fully unlocked. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-06_20-25-51"))
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, with profits, mixed curves. Fully unlocked. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_09-22-04"))
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, no profits, mixed curves. Unlocking with lifetime 5. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)



In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_09-39-00"))
px.line(
    rdf.query("scheduler_metric != 'FlatRelevance'").sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, with profits, mixed curves. Unlocking with lifetime 5. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Tuning
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_10-32-11"))
rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [ ]:
# Tuning
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_14-28-33"))
rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_02-11-12"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_02-25-54"))])
# rdf.head()
# rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    color="metric_recomputation_period",
    log_x=True,
    width=800,
    height=600,
    # range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="10 initial + 20 online blocks, (large) profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Okay I have a good temperature now (0.01). What is a good load?
# WIP on C2
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_02-31-44"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_02-44-34"))])
px.line(
    rdf.sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="10 initial + 20 online blocks, (large) profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Temp tuning with profits
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_14-55-57"))
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-04-36"))

px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-08-13"))

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-18-48"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-52-12"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline.  Poly ratio. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_18-34-39"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_19-32-46"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_20-34-49"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# On privatekube now!!


rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-29-50"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-42-40"))])
px.bar(
    rdf.sort_values(["T", "scheduler_metric"]),
    # x="T",
    x="scheduler_metric",    
    y="realized_profit",
    # color="scheduler_metric",
    # log_x=True,
    width=1000,
    height=600,
    range_y=[0,6_000],
    title="10 initial blocks + 20 online blocks, with profits, PrivateKube. Unlocking with lifetime 10. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-40-36"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-43-03"))])
px.bar(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="scheduler_metric",
    y="n_allocated_tasks",
    # log_x=True,
    width=1000,
    height=600,
    range_y=[0,10_000],
    title="10 initial blocks + 20 online blocks, NO profits, PrivateKube. Unlocking with lifetime 10. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)